# Getting Your Data From Yelp!

In order to make sure you are on track to completing the project, you will complete this workbook first. Below are steps that you need to take in order to make sure you have your data from yelp and are ready to analyze it. Your cohort lead will review this workbook with you the Wednesday before your project is due.    

## Part 1 - Understanding your data and question

You will be pulling data from the Yelp API to complete your analysis. The API, however, provides you with a lot of information that will not be pertinent to your analysis. You will pull data from the API and parse through it to keep only the data that you will need. In order to help you identify that information,look at the API documentation and understand what data the API will provide you. 

Identify which data fields you will want to keep for your analysis. 

https://www.yelp.com/developers/documentation/v3/get_started

## Part 2 - Create ETL pipeline for the business data from the API

### Details

Now that you know what data you need from the API, you want to write code that will execute an API call, parse those results and then insert the results into the DB.  

It is helpful to break this up into three different functions (*API call, parse results, and insert into DB*) and then you can write a function/script that pull the other three functions together. 

Let's first do this for the Business endpoint.

### Request

In [97]:
import requests
import pandas as pd
import json

with open(r'C:\Users\bmcca\.secret\yelp_api.json') as f:
    keys = json.load(f)

client_id = keys['id']
yelp_key = keys['key']

# keys.values()

In [98]:
def yelp_request(term, location, yelp_key, verbose=True):
    '''Adapted from Yelp API Lab: https://github.com/BenJMcCarty/dsc-yelp-api-lab/tree/solution'''
    
    url = 'https://api.yelp.com/v3/businesses/search'

    headers = {
            'Authorization': 'Bearer {}'.format(yelp_key),
        }

    url_params = {
                    'term': term.replace(' ', '+'),
                    'location': location.replace(' ', '+'),
                    'limit': 50
                }
    response = requests.get(url, headers=headers, params=url_params)
    
    if verbose == True:
        print(response)
        print(type(response.text))
        print(response.text[:1000])
        
    return response.json()

In [99]:
response = yelp_request('winery','Southern California', yelp_key)

<Response [200]>
<class 'str'>
{"businesses": [{"id": "4N2pC-h3Ba6qUQ7cN4xP-g", "alias": "san-antonio-winery-ontario", "name": "San Antonio Winery", "image_url": "https://s3-media2.fl.yelpcdn.com/bphoto/__5IBirgkaHoRGdVnUIdng/o.jpg", "is_closed": false, "url": "https://www.yelp.com/biz/san-antonio-winery-ontario?adjust_creative=eyY_X0k6PMB2bu6EM1dXdw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=eyY_X0k6PMB2bu6EM1dXdw", "review_count": 248, "categories": [{"alias": "wineries", "title": "Wineries"}], "rating": 4.0, "coordinates": {"latitude": 34.022224, "longitude": -117.558985}, "transactions": [], "price": "$$", "location": {"address1": "2802 S Milliken Ave", "address2": null, "address3": "", "city": "Ontario", "zip_code": "91761", "country": "US", "state": "CA", "display_address": ["2802 S Milliken Ave", "Ontario, CA 91761"]}, "phone": "+19099473995", "display_phone": "(909) 947-3995", "distance": 26348.268371330654}, {"id": "Q0Cuh5XZY29AdrWpUv8yhw", "alias": 

In [100]:
# Identify keys

response.keys()

dict_keys(['businesses', 'total', 'region'])

In [101]:
# Show first item w/in list of businesses

response['businesses'][0]['coordinates']

{'latitude': 34.022224, 'longitude': -117.558985}

### Parse

#### TODO: Figure out why it's not filtering out null values for Price

In [102]:
# # your code to parse the result to make them easier to insert into the DB

# def parse_results(results):
    
#     return parsed_results

In [109]:
def parse_data(list_of_data):
  
    PD = []
     
    for business in list_of_data:
        if 'price' in business:
            biz_tuple = {'name': business['name'], 'location': business['location']['display_address'], 'id': business['id'], 'rating': business['rating'],
                      'review_count': business['review_count'], 'price': business['price'], 'coordinates': business['coordinates']}
        else:
            biz_tuple = {'name': business['name'], 'location': business['location']['display_address'], 'id': business['id'], 'rating': business['rating'],
                      'review_count': business['review_count'], 'coordinates': business['coordinates']}
    
        PD.append(biz_tuple)
    
    return PD


In [110]:
pd_biz = parse_data(response['businesses'])

In [119]:
# type(pd_biz)

In [112]:
df_pd_biz = pd.DataFrame(pd_biz)

In [113]:
df_pd_biz.dropna(inplace=True)

In [114]:
df_pd_biz.head()

,name,location,id,rating,review_count,price,coordinates
0,San Antonio Winery,"[2802 S Milliken Ave, Ontario, CA 91761]",4N2pC-h3Ba6qUQ7cN4xP-g,4.0,248,$$,"{'latitude': 34.022224, 'longitude': -117.558985}"
1,Joseph Filippi Winery,"[12467 Baseline Rd, Rancho Cucamonga, CA 91739]",Q0Cuh5XZY29AdrWpUv8yhw,4.0,158,$$,"{'latitude': 34.1209259033203, 'longitude': -1..."
2,Galleano Winery,"[4231 Wineville Ave, Mira Loma, CA 91752]",dMy0GL2mG9hKIIMsfAQAgQ,4.0,171,$$,"{'latitude': 34.0109, 'longitude': -117.54246}"
3,Third Street Wine Shop,"[2142 3rd St, La Verne, CA 91750]",Liy2V5TissVcWFt9-45AKg,5.0,101,$$,"{'latitude': 34.09999, 'longitude': -117.76946}"
4,The Claremont Packing House,"[532 W 1st St, Claremont, CA 91711]",jJYkAV0acTPOkqESRWjmTA,4.0,23,$$,"{'latitude': 34.094265, 'longitude': -117.721303}"


### Save

#### TODO: Figure out why it's not checking if the file already exists

In [115]:
# csv_path = r'C:\Users\bmcca\Documents\Flatiron_Stuff\Phase Projects\Phase-1-Project\data\wineries.csv'

In [116]:
def df_save(parsed_results):
       
    import os
    
    if not os.path.isfile(r'C:\Users\bmcca\Documents\Flatiron_Stuff\Phase Projects\Phase-1-Project\data\wineries.csv'):
        parsed_results.to_csv(r'C:\Users\bmcca\Documents\Flatiron_Stuff\Phase Projects\Phase-1-Project\data\wineries.csv', header='column_names')
    else: # else it exists so append without writing the header
        parsed_results.to_csv(r'C:\Users\bmcca\Documents\Flatiron_Stuff\Phase Projects\Phase-1-Project\data\wineries.csv', mode='a', header=False)
    return

In [117]:
df_save(df_pd_biz)

### Condense into one function

- Write a script that combines the three functions above into a single process.

While it will take some experimentation to write the functions above, once you get them working it will be best to put them in a `.py` file and then import the functions to use in a script 

In [ ]:
from helper_funcs import *

In [ ]:


# create a variable  to keep track of which result you are in. 
cur = 0

#set up a while loop to go through and grab the result 
while cur < num and cur < 1000:
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    #make your API call with the new offset number
    results = yelp_call(url_params, api_key)
    
    #after you get your results you can now use your function to parse those results
    parsed_results = parse_results(results)
    
    # use your function to insert your parsed results into the db
    df_save(parsed_results)
    #increment the counter by 50 to move on to the next results
    cur += 20

___

## Part 3 -  Create ETL pipeline for the restaurant review data from the API

You've done this for the Businesses, now you need to do this for reviews. You will follow the same process, but your functions will be specific to reviews. Above you ahve a model of the functions you will need to write, and how to pull them together in one script. For this part, you ahve the process below 

- In order to pull the reveiws, you will need the business ids. So your first step will be to get all of the business ids from your businesses csv. 

- Write a function that takes a business id and makes a call to the API for reivews


- Write a function to parse out the relevant information from the reviews

- Write a function to save the parse data into a csv file containing all of the reviews. 

- Combine the functions above into a single script  

___

## Part 4 -  Using python and pandas, write code to answer the questions below. 

- Which are the 5 most reviewed businesses in your dataset?
- What is the highest rating recieved in your data set and how many businesses have that rating?
- What percentage of businesses have a rating greater than or  4.5?
- What percentage of businesses have a rating less than 3?
- What percentage of your businesseshave a price label of one dollar sign? Two dollar signs? Three dollar signs? No dollar signs?
- Return the text of the reviews for the most reviewed business. 
- Find the highest rated business and return text of the most recent review. If multiple business have the same rating, select the business with the most reviews. 
- Find the lowest rated business and return text of the most recent review.  If multiple business have the same rating, select the business with the least reviews. 

___

# Reference help

###  Pagination

Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. 

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and APIs can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

***Below is sample code that you can use to help you deal with the pagination parameter and bring all of the functions together.***


***Also, something might cause your code to break while it is running. You don't want to constantly repull the same data when this happens, so you should insert the data into the database as you call and parse it, not after you have all of the data***


In [ ]:
# create a variable  to keep track of which result you are in. 
cur = 0

#set up a while loop to go through and grab the result 
while cur < num and cur < 1000:
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    #make your API call with the new offset number
    results = yelp_call(url_params, api_key)
    
    #after you get your results you can now use your function to parse those results
    parsed_results = parse_results(results)
    
    # use your function to insert your parsed results into the db
    db_insert(parsed_results)
    #increment the counter by 50 to move on to the next results
    cur += 20